# Classification-1

2値分類用のサンプルデータである乳がんデータを読み込みます。<br>特徴量が全て数値データで欠損値もないキレイなサンプルデータです。

In [ ]:
# Load and return the breast cancer wisconsin dataset (classification).
# The breast cancer dataset is a classic and very easy binary classification dataset.
import pandas as pd
from sklearn.datasets import load_breast_cancer

# set data by role
dataset = load_breast_cancer()
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = pd.Series(dataset.target, name='y')

# check the shape
print('--------------------------------------------------')
print('X shape: (%i,%i)' %X.shape)
print('--------------------------------------------------')
print(y.value_counts())
print('--------------------------------------------------')
print('y=0 means Marignant(悪性),y=1 means Benign(良性):')
print('--------------------------------------------------')
X.join(y).head()

次に評価指標についてです。回帰モデルの評価にはR2スコアなどがありました。分類モデルの評価の場合、最もわかりやすいものが正解率（Accuracy）です。正解率は予測対象データ（スコアリングデータ）の全件をNとした時、正例を正しく正例と分類できた数TP、負例を正しく負例と分類できた数TFとすると、 以下の数式で表せます。<br><center>Accuracy = (TP + TF) / N</center><br>つまり、正例・負例を問わず正しく分類されたサンプル数の割合です。sklearnでは正解データと予測データを与えると、Accuracyを計算してくれる関数が存在します。以下で呼び出しておきます。

In [ ]:
# 分類モデルの評価指標計算のための関数の読込
from sklearn.metrics import accuracy_score

下記コードは回帰と同様ですので、各行の役割を理解できていることを確認して下さい。

In [ ]:
# import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Holdout
X_train,X_test,y_train,y_test = train_test_split(X,
                                                 y,
                                                 test_size=0.20,
                                                 random_state=1)
# set pipelines for two different algorithms
pipelines ={
    'knn': Pipeline([('scl',StandardScaler()),
                       ('est',KNeighborsClassifier())]),

    'logistic': Pipeline([('scl',StandardScaler()),
                          ('est',LogisticRegression(random_state=1))])    
}

# fit the models
for pipe_name, pipeline in pipelines.items():
    pipeline.fit(X_train,y_train)
    print(pipe_name, ': Fitting Done')

学習を終えたので、２つのパイプラインの分類器としての性能（正解率）を比較します。<br>まず、pipelineのpredictメソッドによる出力ベクトルを見て下さい。<br>y_pred in {0,1}で予測ラベルが付与されていることがわかります。

In [ ]:
pipelines['logistic'].predict(X_train)[:10]

ちなみに、今回は使いませんが、実務では予測ラベルの他に各クラスに分類される予測確率をよく使います。<b>予測確率を出力させたい場合はpredictをpredic_probaに変更します。</b>最初の列はy=0と分類される予測確率、2番目の列はy=1と分類される予測確率です。

In [ ]:
pipelines['logistic'].predict_proba(X_train)[:10]

さて、accuracy_scoreは第一引数に正解ラベル、第二引数に予測ラベル（確率ではない）を指定します。ですので、今回はpredictメソッドで予測ラベルを与えましょう。

In [ ]:
scores = {}
for pipe_name, pipeline in pipelines.items():
    scores[(pipe_name,'train')] = accuracy_score(y_train, pipeline.predict(X_train))
    scores[(pipe_name,'test')] = accuracy_score(y_test, pipeline.predict(X_test))

pd.Series(scores).unstack()

このデータに対しては、kNNよりもロジスティック回帰による精度（正解率）の方が勝ることを確認できました。アルゴリズムの知識を増やして、様々なパイプラインを設計できるようになりましょう。